In [1]:
import pandas as pd
import os
import pandas as pd
from allennlp_models import pretrained
from group9_SRL import predict_total
import re
import string as string_value
# from predict import predict_SRL, predict_SRL_BERT, predict_srl_group9
import predict as pred
sentence = "I am a student"
allen_models = ["structured-prediction-srl-bert", 'structured-prediction-srl']

MODEL_SRL_BERT = pretrained.load_predictor(allen_models[0])
MODEL_SRL = pretrained.load_predictor(allen_models[1])

error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\coref-spanbert.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\evaluate_rc-lerc.json as plain json
lerc is not a registered model.
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\generation-bart.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\glove-sst.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\lm-masked-language-model.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\mod

In [17]:
import os
import pandas as pd
from allennlp_models import pretrained
from group9_SRL import predict_total
import re
import string as string_value



def get_constituent(predicate, srl_output):
    
    # get the description of the srl out_put corresponding to the predicate

    for verb in srl_output['verbs']:
        if verb['verb'] == predicate:
            description = verb['description']
            return get_constituent_list(description)
    
            


def get_constituent_list(string):

    # define a regular expression pattern to match the relevant words
    pattern = r'\[(ARG\d+|V):\s*([^\]]+)\]|(\b\w+\b)'

    # find all matches of the pattern in the string
    matches = re.findall(pattern, string)

    # extract the relevant words from the matches and store them in a list
    words = [match[1] or match[2] for match in matches]

    # append the final period to the list
    if string[-1] in string_value.punctuation:

        words.append(string[-1])

    return words    


def parse_string(input_string):
    """
    Parses the string output of the SRL model into a dictionary
    """
    # define a regular expression pattern to match the string format
    pattern = r"\[(\w+): ([^]]+)\]"
        # search for matches using the pattern
    matches = re.findall(pattern, input_string)
    
    # convert the matches into a dictionary
    output_dict = {}
    for match in matches:
        key = match[0]
        value = match[1]
        output_dict[key] = value
    
    return output_dict

def classify(predict, gold):
    """
    Classifies the prediction as correct or incorrect
    input: predict, gold - dictionaries, with keys as labels and values as constituents
    output: total missed, total correct, total incorrect 
    """
    correct = 0
    incorrect = 0
    missed = 0

    for key in gold.keys():
        if key in predict.keys():
            if gold[key].lower() == predict[key].lower():
                correct += 1
            else:
                incorrect += 1
        else:
            missed += 1
    if incorrect > 0:
        return 0
    
    elif missed > 0:
        return 0

    else:
        return 1

def get_description(verb, srl_output):
    """
    Returns the description of the verb
    """
    for verb1 in srl_output['verbs']:
        
        if verb1['verb'] == verb:
            return verb1['description']
        
    print(srl_output)
    return verb1['verb'] # Not correct, but its a temporary fix

def map_list_to_constituent(constituent, output_group_9):
    """
    Maps the output of group 9 to the constituent list
    """
    output_dict = {}

    for i, element in enumerate(output_group_9[1]):
        if element != "_":
            classified_at, label  = output_group_9[0][i], output_group_9[1][i]
            for element1 in constituent:
                if classified_at in element1:
                    output_dict[label] = element1

    return output_dict


def calculate_failure(total, correct):
    """
    Calculates the failure rate
    """
    return (total - correct)/total

def predict_SRL_BERT(text, gold):
    
    predicted_outputs = MODEL_SRL_BERT.predict(sentence = text)
    description = get_description(gold, predicted_outputs)
    return get_constituent_list(description), parse_string(description)




def predict_SRL(text, gold):
        
    predicted_outputs = MODEL_SRL.predict(sentence = text)

    
    return parse_string(get_description(gold, predicted_outputs))


def predict_srl_group9(text, constituent    ):

    predicted_outputs = predict_total(text)

    return map_list_to_constituent(constituent, predicted_outputs)


In [18]:
def test_five():
        
    """
    Testing SRL capability on handeling, Verb Alternations/Levin's classes (DIR)
    """
    df = pd.read_json("testdata/test5.json")
    tests = df['tests']
    total_srl, total_srl_bert, total_group_9 = 0, 0, 0
    for test in tests:

        causative = test['causative']
        inchoative = test['inchoative']

        golden_labels = test['propbank_golden_labels']

        causative_gold = golden_labels['causative']
        inchoative_gold = golden_labels['inchoative']

        causative_pred = causative_gold['V']
        inchoative_pred = inchoative_gold['V']


        constituent, output_causative_srl_bert = predict_SRL_BERT(causative, causative_pred)
        output_causative_srl = predict_SRL(causative, causative_pred)
        output_causative_group_9 = pred.predict_srl_group9(causative, constituent)


        constituent, output_inchoative_srl_bert = predict_SRL_BERT(inchoative, inchoative_pred)
        output_inchoative_srl = predict_SRL(inchoative, inchoative_pred)
        output_inchoative_group_9 = pred.predict_srl_group9(inchoative, constituent)


        SRL_score = classify(output_causative_srl, causative_gold) * classify(output_inchoative_srl, inchoative_gold)

        SRL_BERT_score = classify(output_causative_srl_bert, causative_gold) * classify(output_inchoative_srl_bert, inchoative_gold)

        SRL_group_9_score = classify(output_causative_group_9, causative_gold) * classify(output_inchoative_group_9, inchoative_gold)


        total_srl += SRL_score
        total_srl_bert += SRL_BERT_score
        total_group_9 += SRL_group_9_score

        

    print("SRL score: ", total_srl)
    print("SRL BERT score: ", total_srl_bert)
    print("SRL Group 9 score: ", total_group_9)

    return calculate_failure(len(tests), total_srl), calculate_failure(len(tests), total_srl_bert), calculate_failure(len(tests), total_group_9)

test_five()

SRL score:  8
SRL BERT score:  7
SRL Group 9 score:  6


(0.1111111111111111, 0.2222222222222222, 0.3333333333333333)

In [3]:
def test_eight():
    """
    Testing the SRL model on the test data for spelling errors
    """
    df = pd.read_json("testdata/test8.json")
    tests = df['tests']

    for test in tests:
        sentence = test['error_sentence']
        golden_labels = test['propbank_golden_labels']

        predicate = golden_labels['V']

        try:
            constituent, output_srl_bert = predict_SRL_BERT(sentence, predicate)

            output_srl = predict_SRL(sentence, predicate)
            output_group_9 = predict_srl_group9(sentence, constituent)
        except:
            print("Error in sentence: ", sentence)
            continue

        print(output_srl_bert, golden_labels)

        print(classify(output_srl_bert, golden_labels))
        print(classify(output_srl, golden_labels))
        print(classify(output_group_9, golden_labels))


test_eight()

{'verbs': [], 'words': ['The', 'boy', 'at', 'an', 'aplle', '.']}
Error in sentence:  The boy at an aplle.
{'ARG0': 'She', 'V': 'bot', 'ARG1': 'a red dres'} {'ARG0': 'She', 'V': 'bot', 'ARG1': 'a red dres'}
1
1
1
{'ARG0': 'He', 'V': 'loves', 'ARG1': 'to ply basketball'} {'ARG0': 'He', 'V': 'loves', 'ARG1': 'to ply basketball'}
1
1
0
{'verbs': [], 'words': ['The', 'kat', 'chasd', 'the', 'mose', '.']}
Error in sentence:  The kat chasd the mose.
{'ARG0': 'The teecher', 'V': 'gradded', 'ARG1': 'the exms'} {'ARG0': 'The teecher', 'V': 'gradded', 'ARG1': 'the exms'}
1
1
0
{'verbs': [], 'words': ['The', 'doog', 'chasd', 'the', 'kat', '.']}
Error in sentence:  The doog chasd the kat.
{'ARG0': 'The mn', 'V': 'roed', 'ARG1': 'his bke'} {'ARG0': 'The mn', 'V': 'roed', 'ARG1': 'his bke to work'}
0
0
0
{'ARG0': 'She', 'V': 'sang', 'ARG1': 'a butiful sonng'} {'ARG0': 'She', 'V': 'sang', 'ARG1': 'a butiful sonng'}
1
1
1
{'ARG0': 'The boy', 'V': 'kiked', 'ARG1': 'the bal'} {'ARG0': 'The boy', 'V': 'kik

In [4]:
def test_seven():
    """
    Testing the SRL model on the test data for Polysemy - Lexical Semantics
    """
    df = pd.read_json("testdata/test7.json")
    tests = df['tests']

    for test in tests:
        sentence = test['sentence']
        golden_labels = test['propbank_golden_labels']
        print(golden_labels)
        predicate = golden_labels['V']
        print(predicate)
        constituent, output_srl_bert = predict_SRL_BERT(sentence, predicate)

        output_srl = predict_SRL(sentence, predicate)
        output_group_9 = predict_srl_group9(sentence, constituent)

        print(output_srl_bert, golden_labels)

        print(classify(output_srl_bert, golden_labels))
        print(classify(output_srl, golden_labels))
        print(classify(output_group_9, golden_labels))


# test_seven()

In [11]:
def test_four():
    """
    Testing the SRL model on the test data for Cleft sentences
    """
    df = pd.read_json("testdata/test4.json")
    tests = df['tests']

    for test in tests:
        sentence = test['sentence']
        golden_labels = test['propbank_golden_labels']

        predicate = golden_labels['V']
        constituent, output_srl_bert = predict_SRL_BERT(sentence, predicate)

        output_srl = predict_SRL(sentence, predicate)
        output_group_9 = predict_srl_group9(sentence, constituent)

        print(output_srl_bert, golden_labels)

        print(classify(output_srl_bert, golden_labels))
        print(classify(output_srl, golden_labels))
        print(classify(output_group_9, golden_labels))


# test_four()

In [19]:
def test_one():
    """
    Testing SRL capability on handeling, statement -> multiple realizations
    """
    
    df = pd.read_json("testdata/test1.json")
    tests = df['tests']
    total_srl, total_srl_bert, total_group_9 = 0, 0, 0
    for test in tests[:]:
        original = test['proposition']
        realizations = test['realizations']
        real_srl, real_srl_bert, real_group_9 = 1, 1, 1
        for real in realizations:
            sentence = real['realization']
            gold_label = real['propbank_golden_labels']
          
            predicate_gold = gold_label['V']

            
            constituent, output_srl_bert = predict_SRL_BERT(sentence, predicate_gold)
            output_srl = predict_SRL(sentence, predicate_gold)

            output_group_9 = predict_srl_group9(sentence, constituent)
            

            real_srl *= classify(output_srl, gold_label)
            real_srl_bert *= classify(output_srl_bert, gold_label)
            real_group_9 *= classify(output_group_9, gold_label)

        total_srl += real_srl
        total_srl_bert += real_srl_bert
        total_group_9 += real_group_9

    print("SRL score: ", total_srl)
    print("SRL BERT score: ", total_srl_bert)
    print("SRL Group 9 score: ", total_group_9)

    return calculate_failure(len(tests), total_srl), calculate_failure(len(tests), total_srl_bert), calculate_failure(len(tests), total_group_9)

test_one()

SRL score:  2
SRL BERT score:  2
SRL Group 9 score:  0


(0.7142857142857143, 0.7142857142857143, 1.0)

In [21]:
def test_two():
        
    """
    Testing SRL capability on handeling, statement -> Question
    """
    # TODO check if predicted is correct
    df = pd.read_json("testdata/test2.json")
    tests = df['tests']
    total_srl, total_srl_bert, total_group_9 = 0, 0, 0
    for test in tests:

        statement = test['statement']
        question = test['question']

        golden_labels = test['propbank_golden_labels'][0]

        statement_gold = golden_labels['Statement']
        question_gold = golden_labels['Question']

        statement_pred = statement_gold['V']
        question_pred = question_gold['V']


        constituent, output_statement_srl_bert = predict_SRL_BERT(statement, statement_pred)
        output_statement_srl = predict_SRL(statement, statement_pred)
        output_statement_group_9 = pred.predict_srl_group9(statement, constituent)


        constituent, output_question_srl_bert = predict_SRL_BERT(question, question_pred)

        output_question_srl = predict_SRL(question, question_pred)
        output_question_group_9 = pred.predict_srl_group9(question, constituent)

        SRL_score = classify(output_statement_srl, statement_gold) * classify(output_question_srl, question_gold)

        SRL_BERT_score = classify(output_statement_srl_bert, statement_gold) * classify(output_question_srl_bert, question_gold)

        SRL_group_9_score = classify(output_statement_group_9, statement_gold) * classify(output_question_group_9, question_gold)

        
        total_srl += SRL_score
        total_srl_bert += SRL_BERT_score
        total_group_9 += SRL_group_9_score

    print("SRL score: ", total_srl)
    print("SRL BERT score: ", total_srl_bert)
    print("SRL Group 9 score: ", total_group_9)

    return calculate_failure(len(tests), total_srl), calculate_failure(len(tests), total_srl_bert), calculate_failure(len(tests), total_group_9)


test_two()

{'verbs': [{'verb': 'Is', 'description': '[V: Is] the infant snoozing in the cradle ?', 'tags': ['B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}], 'words': ['Is', 'the', 'infant', 'snoozing', 'in', 'the', 'cradle', '?']}
{'verbs': [{'verb': 'Is', 'description': '[V: Is] [ARG1: the infant snoozing] [ARG2: in the cradle] ?', 'tags': ['B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']}], 'words': ['Is', 'the', 'infant', 'snoozing', 'in', 'the', 'cradle', '?']}
SRL score:  3
SRL BERT score:  3
SRL Group 9 score:  0


(0.5714285714285714, 0.5714285714285714, 1.0)

In [ ]:
def test_three():
    """
    Testing SRL capabiltie of handeling, active -> passive
    """
    df = pd.read_json("testdata/test3.json")
    tests = df['tests']
    srl, bert, group9 = 0, 0, 0
    for test in tests:
        active = test['active']
        passive = test['passive']

        golden_labels = test['propbank_golden_labels'][0]

        active_gold = golden_labels['Active']
        passive_gold = golden_labels['Passive']

        active_predicate = active_gold['V']
        passive_predicate = passive_gold['V']



        constituent, output_active_srl = predict_SRL_BERT(active, active_predicate)
        output_active_srl_bert = predict_SRL(active, active_gold)
        output_active_group_9 = predict_srl_group9(active, constituent)


        constituent, output_passive_srl = predict_SRL_BERT(passive, passive_predicate)
        output_passive_srl_bert = predict_SRL(passive, passive_predicate)
        output_passive_group_9 = predict_srl_group9(passive, constituent)

        print(output_active_srl, output_passive_srl,)
        print(classify(output_active_srl, active_gold), classify(output_passive_srl, passive_gold), active_gold, passive_gold)
        # print(output_active_srl_bert,  "\n\n", active_gold)
        srl += (classify(output_active_srl, active_gold) * classify(output_passive_srl, passive_gold))
        bert += (classify(output_active_srl_bert, active_gold) * classify(output_passive_srl_bert, passive_gold))
        group9 += (classify(output_active_group_9, active_gold) * classify(output_passive_group_9, passive_gold))

    print(srl, bert)
# test_three()

In [12]:
def test_four():
    """
    Testing the SRL model on the test data for Cleft sentences
    """
    df = pd.read_json("testdata/test4.json")
    tests = df['tests']

    for test in tests:
        sentence = test['sentence']
        golden_labels = test['propbank_golden_labels']

        predicate = golden_labels['V']
        constituent, output_srl_bert = predict_SRL_BERT(sentence, predicate)

        output_srl = predict_SRL(sentence, predicate)
        output_group_9 = predict_srl_group9(sentence, constituent)

        print(output_srl_bert, golden_labels)

        print(classify(output_srl_bert, golden_labels))
        print(classify(output_srl, golden_labels))
        print(classify(output_group_9, golden_labels))


# test_four()

In [ ]:
def test_five():
        
    """
    Testing SRL capability on handeling, Verb Alternations/Levin's classes (DIR)
    """
    df = pd.read_json("testdata/test5.json")
    tests = df['tests']
    total_srl, total_srl_bert, total_group_9 = 0, 0, 0
    for test in tests:

        causative = test['causative']
        inchoative = test['inchoative']

        golden_labels = test['propbank_golden_labels']

        causative_gold = golden_labels['causative']
        inchoative_gold = golden_labels['inchoative']

        causative_pred = causative_gold['V']
        inchoative_pred = inchoative_gold['V']


        constituent, output_causative_srl_bert = predict_SRL_BERT(causative, causative_pred)
        output_causative_srl = predict_SRL(causative, causative_pred)
        output_causative_group_9 = pred.predict_srl_group9(causative, constituent)


        constituent, output_inchoative_srl_bert = predict_SRL_BERT(inchoative, inchoative_pred)
        output_inchoative_srl = predict_SRL(inchoative, inchoative_pred)
        output_inchoative_group_9 = pred.predict_srl_group9(inchoative, constituent)


        SRL_score = classify(output_causative_srl, causative_gold) * classify(output_inchoative_srl, inchoative_gold)

        SRL_BERT_score = classify(output_causative_srl_bert, causative_gold) * classify(output_inchoative_srl_bert, inchoative_gold)

        SRL_group_9_score = classify(output_causative_group_9, causative_gold) * classify(output_inchoative_group_9, inchoative_gold)


        total_srl += SRL_score
        total_srl_bert += SRL_BERT_score
        total_group_9 += SRL_group_9_score

        

    print("SRL score: ", total_srl)
    print("SRL BERT score: ", total_srl_bert)
    print("SRL Group 9 score: ", total_group_9)

    return calculate_failure(len(tests), total_srl), calculate_failure(len(tests), total_srl_bert), calculate_failure(len(tests), total_group_9)

test_five()

In [17]:
def test_six():
    """
    Testing SRL capabiltie of handeling, active -> passive
    """
    df = pd.read_json("testdata/test6.json")
    tests = df['tests']
    srl, bert, group9 = 0, 0, 0
    for test in tests:
        sentence = test['sentence']
        synonym = test['synonym_sentence']
        golden_labels = test['propbank_golden_labels']

        sentence_gold = golden_labels['Sentence']
        synonym_gold = golden_labels['SynonymSentence']

        sentence_predicate = sentence_gold['V']
        synonym_predicate = synonym_gold['V']



        constituent, output_sentence_srl_bert = predict_SRL_BERT(sentence, sentence_predicate)
        output_sentence_srl = predict_SRL(sentence, sentence_predicate)
        output_sentence_group_9 = predict_srl_group9(sentence, constituent)


        constituent, output_synonym_srl_bert = predict_SRL_BERT(synonym, synonym_predicate)
        output_synonym_srl = predict_SRL(synonym, synonym_predicate)
        output_synonym_group_9 = predict_srl_group9(synonym, constituent)

        print(output_sentence_srl, output_synonym_srl)
        print(output_sentence_srl_bert, output_synonym_srl_bert)

        print(output_sentence_group_9, output_synonym_group_9)

        srl += (classify(output_sentence_srl, sentence_gold) * classify(output_synonym_srl, synonym_gold))
        bert += (classify(output_sentence_srl_bert, sentence_gold) * classify(output_synonym_srl_bert, synonym_gold))
        group9 += (classify(output_sentence_group_9, sentence_gold) * classify(output_synonym_group_9, synonym_gold))


    print(srl, bert, group9)
# test_six()

In [ ]:
def test_seven():
    """
    Testing the SRL model on the test data for Polysemy - Lexical Semantics
    """
    df = pd.read_json("testdata/test7.json")
    tests = df['tests']

    for test in tests:
        sentence = test['sentence']
        golden_labels = test['propbank_golden_labels']
        print(golden_labels)
        predicate = golden_labels['V']
        print(predicate)
        constituent, output_srl_bert = predict_SRL_BERT(sentence, predicate)

        output_srl = predict_SRL(sentence, predicate)
        output_group_9 = predict_srl_group9(sentence, constituent)

        print(output_srl_bert, golden_labels)

        print(classify(output_srl_bert, golden_labels))
        print(classify(output_srl, golden_labels))
        print(classify(output_group_9, golden_labels))


# test_seven()